![](../../PythonCompendium/storage/banners/28_observability.png)

##### **<u>Свои догадки</u>**

###### <u>Попытка вывода симметрии у решения</u> - Частные случаи

In [11]:
from sympy import *
import sympy
from cosmetic import *

def add_extraname(s, extraname):
    return " ".join([i+extraname for i in s.split()])

def get_x_y_z(case: dict, extraname: str = "", planar: bool = False, test_xz: bool = False):
    x0, y0, z0, vx0, vy0, vz0 = var(add_extraname(s="x_0 y_0 z_0 v^x_0 v^y_0 v^z_0", extraname=extraname))
    X0, Y0, Z0, VX0, VY0, VZ0 = var(add_extraname(s="X_0 Y_0 Z_0 V^X_0 V^Y_0 V^Z_0", extraname=extraname))
    c, rho, m, M, s, S, v_orb = var(add_extraname(s="C rho m M s S v_{orb}", extraname=extraname))
    t, w0 = var("t w_0")

    r, R = 3*[None], 3*[None]
    
    if case['C1 = 0']:
        vx0 = - 2 * z0 * w0
        VX0 = - 2 * Z0 * w0
    if not case['CubeSat motion']:
        X0, Y0, Z0, VX0, VY0, VZ0 = [0]*6
    if planar:
        y0, Y0, vy0, VY0 = [0]*4

    for _x0, _y0, _z0, _vx0, _vy0, _vz0, _r, _m, _s in zip((x0, X0), (y0, Y0), (z0, Z0), (vx0, VX0), (vy0, VY0), (vz0, VZ0), 
                                                           (r, R), (m, M), (s, S)):
        C_1 = 2 * _z0 + _vx0 / w0
        C_2 = _vz0 / w0
        C_3 = -3 * _z0 - 2 * _vx0 / w0
        C_4 = _x0 - 2 * _vz0 / w0
        C_5 = _vy0 / w0
        C_6 = _y0
        
        tmp = (-t**2 / 2 * c * rho / _m * _s * v_orb**2) if case['Aero'] else 0
        
        _r[0] = C_4 - 3*C_1*w0*t + 2*C_2*cos(w0*t) -2*C_3*sin(w0*t) + tmp
        _r[1] = C_5*sin(w0*t) + C_6*cos(w0*t)
        _r[2] = 2*C_1 + C_2*sin(w0*t) + C_3*cos(w0*t)
    
    return {'w0': w0, 't': t,
            'x0': x0, 'y0': y0, 'z0': z0, 'vx0': vx0, 'vy0': vy0, 'vz0': vz0,
            'x': r[0], 'y': r[1], 'z': r[2], 
            'X': R[0], 'Y': R[1], 'Z': R[2]}

def get_measurements(case: dict, params: dict):
    if case['antenna type'] == 'isotropic':
        measurements = [(params['X'] - params['x'])**2 + 
                        (params['Y'] - params['y'])**2 + 
                        (params['Z'] - params['z'])**2]

    return measurements

def get_discrepancy_invert_param(measurements, params: dict, variables: list) -> list:
    global counter
    subses = []
    for v in variables:
        subses.append((params[v], -params[v]))

    anw = []
    for i, m in zip(range(len(measurements)), measurements):
        tmp = (m - m.subs(subses)).simplify()
        anw.append(tmp)
        if isinstance(tmp, sympy.core.numbers.Zero):
            print(f"По параметрам\033[1m", *variables, f"\033[0mу измерения №{i+1} есть зеркальная симметрия")
            counter *= 2
    return anw

def print_symmetry_problem_result():
    global counter
    if counter == 1:
        my_print(f"Решение единственное: {counter}!", bold=True, color="g")
    else:
        my_print(f"Симметричных решений: {counter}", bold=True, color="b")


def calculate_symmetry(motion_case, measurement_case, v_list):
    global counter
    counter = 1
    params = get_x_y_z(case=motion_case)
    measurements = get_measurements(case=measurement_case, params=params)
    for variables in v_list:
        _ = get_discrepancy_invert_param(measurements=measurements, variables=variables, params=params)
    print_symmetry_problem_result()

###### 0. Поиск решения

>> Уравнения:
$$\ddot{x} = -2 \omega_0 \upsilon_z,$$
$$\ddot{y} = -\omega_0^2 y,$$
$$\ddot{z} = 2 \omega_0 \upsilon_x + 3 \omega_0^2 z.$$

> Движение по $y$ отделимо <br>
> Возможна симметрия по одновременному изменению положений и скоростей по $x, z$ <br>

In [13]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': False,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

params = get_x_y_z(case=motion_case)
measurements = get_measurements(case=measurement_case, params=params)

# params_wrong = get_x_y_z(case=motion_case, planar=False, extraname="^w")
# measurements_wrong = get_measurements(case=measurement_case, params=params_wrong)
measurements[0]

(-v^y_0*sin(t*w_0)/w_0 - y_0*cos(t*w_0))**2 + (-2*v^x_0/w_0 - v^z_0*sin(t*w_0)/w_0 - 4*z_0 - (-2*v^x_0/w_0 - 3*z_0)*cos(t*w_0))**2 + (t*w_0*(3*v^x_0/w_0 + 6*z_0) - 2*v^z_0*cos(t*w_0)/w_0 + 2*v^z_0/w_0 - x_0 + (-4*v^x_0/w_0 - 6*z_0)*sin(t*w_0))**2

In [16]:
grad_vec = {}
for v in ["x0", "y0", "z0", "vx0", "vy0", "vz0"]:
    grad_vec[v] = measurements[0].diff(params[v])

###### 1. <u>ХКУ, 1к-1ч, изотропные антенны, без определение углового движения</u>

> Уравнения ХКУ имеют 4 симметричных решения <br>
> Антенны изотропные, но углы не ищутся -> симметрия не уменьшается <br>
> Итого: 4 симметричных решения

In [117]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': False,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам y0 vy0 у измерения №1 есть зеркальная симметрия
По параметрам x0 z0 vx0 vz0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 4


In [118]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': False,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

Решение единственное: 1!


In [119]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': False,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть зеркальная симметрия
По параметрам y0 vy0 у измерения №1 есть зеркальная симметрия
По параметрам x0 z0 vz0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 8


In [120]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': False,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 2


###### 2. <u>ХКУ + аэро, 1к-1ч, изотропные антенны, без определение углового движения</u>

> Уравнения ХКУ имеют 4 симметричных решения <br>
> Антенны изотропные, но углы не ищутся -> симметрия не уменьшается <br> 
> Итого: 4 симметричных решения

In [121]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': True,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам y0 vy0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 2


In [122]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': True,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)


KeyboardInterrupt



In [105]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': True,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть симметрия
По параметрам y0 vy0 у измерения №1 есть симметрия
Симметричных решений: 4


In [106]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': True,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть симметрия
Симметричных решений: 2


###### <u>Минимальное кол-во материнских аппаратов для наблюдаемости</u>

In [9]:
def get_min_chief_amount(fn: int, motion: str, antenna_config: dict) -> int:
    """
    :param fn: кол-во дочерних КА (НЕ УЧИТЫВАЕТСЯ)
    :param motion: По каким координатам ОСК движение КА
    :param antenna_config: типы антенн материнских и дочерних КА
    :return: Кол-во материнских КА для наблюдаемости на НОО (околокруговые)
    """
    for cn in range(1, 10):
        n_symmetry_solution = [1, 1, 1]
        ############################
        # Увеличение симметричных решений
        if antenna_config['c-type'] == "изотропные":
            for i in range(3):
                n_symmetry_solution[i] *= 2
        if antenna_config['d-type'] == "изотропные":
            for i in range(3):
                n_symmetry_solution[i] *= 2
        
        # Уменьшение симметричных решений
        if not antenna_config['c-multy-send'] and not antenna_config['d-multy-take']:
            if cn == 4:
                for i in range(3):
                    n_symmetry_solution[i] /= 2
        ############################
        if sum(n_symmetry_solution) == 3:
            return cn
    print("Не нашлось такого количества! Измени параметры")
    return -1
    
cn = get_min_chief_amount(fn=1, motion="xyz", 
                          antenna_config={'c-type': ["изотропные"][0], 
                                          'd-type': ["изотропные"][0],
                                          'c-multy-send': False,
                                          'c-multy-take': False,
                                          'd-multy-send': False,
                                          'd-multy-take': False,})
print(f"Минимальное кол-во материнских аппаратов для наблюдаемости: \033[1m{cn}\033[0m")

Не нашлось такого количества! Измени параметры
Минимальное кол-во материнских аппаратов для наблюдаемости: -1


##### **<u>RSSI to range-based</u>**

In [1]:
from common_func import *

Инициализация проекта kiam-formation


In [2]:
r = Matrix(var('r_x r_y r_z'))
xr, yr, zr = var('x_r y_r z_r')
dx, dy, dz = var('δx δy δz')
q = kf.vec2quat(Matrix(var('q_x q_y q_z')))

###### **ВЫКЛАДКИ ДЛЯ ПОЛУВОЛНОВОЙ АНТЕННЫ** (я легко и непринуждённо забил на то, что от $r$ тоже идёт $\delta x$

In [ ]:
series(x, r[0], r[0] + dx, n=6)

In [7]:
d = var('d')
x = eval(str(kf.local_dipole(None, r, 'x')).replace('r_x**2 + r_y**2 + r_z**2', 'd**2')).simplify()
y = eval(str(kf.local_dipole(None, r, 'y')).replace('r_x**2 + r_y**2 + r_z**2', 'd**2')).simplify()
z = eval(str(kf.local_dipole(None, r, 'z')).replace('r_x**2 + r_y**2 + r_z**2', 'd**2')).simplify()
print('Усиление G вдоль оси x:')
display(x)
print('Усиление G вдоль оси y:')
display(y)
print('Усиление G вдоль оси z:')
display(z)

Усиление G вдоль оси x:


1.0*cos(pi*r_x/(2*sqrt(d**2)))/sqrt((r_y**2 + r_z**2)/d**2)

Усиление G вдоль оси y:


1.0*cos(pi*r_y/(2*sqrt(d**2)))/sqrt((r_x**2 + r_z**2)/d**2)

Усиление G вдоль оси z:


1.0*cos(pi*r_z/(2*sqrt(d**2)))/sqrt((r_x**2 + r_y**2)/d**2)

$$Y_x(x) = \frac{\cos\left( \frac{\pi x}{2 r} \right)}{\sqrt{1 - \frac{x^2}{r^2}}}$$

$$Y_x(x + \delta x) = \frac{\cos\left( \frac{\pi x  + \pi \delta x}{2 r} \right)}{\sqrt{1 - \frac{x^2}{r^2}  + \frac{2x\delta x}{r^2}}}$$

Воспользовавшись разложением

$$\frac{1}{\sqrt{1 - \frac{x^2}{r^2}  + \frac{2x\delta x}{r^2}}} = \frac{1}{\sqrt{a  + b \delta x}} = \frac{1}{\sqrt{a}\sqrt{1  + \frac{b}{a} \delta x}}  \approx \frac{1}{\sqrt{a}} \left( 1 - \frac{b\delta x}{2 a} \right) $$

А если мы ещё вспомним про
$$\cos\left( \frac{\pi x}{2 r} + \frac{\pi \delta x}{2 r} \right) = \cos{\left( \frac{\pi x}{2 r} \right)}\cos{\left( \frac{\pi \delta x}{2 r} \right)} - \sin{\left( \frac{\pi x}{2 r} \right)}\sin{\left( \frac{\pi \delta x}{2 r} \right)} \approx$$

$$\approx  \cos{\left( \frac{\pi x}{2 r} \right)} - \sin{\left( \frac{\pi x}{2 r} \right)}\frac{\pi \delta x}{2 r} $$

В конце получим вот такую штуку
$$Y_x(x + \delta x) = \frac{\cos{\left( \frac{\pi x}{2 r} \right)} - \sin{\left( \frac{\pi x}{2 r} \right)}\frac{\pi \delta x}{2 r}}{\sqrt{1 - \frac{x^2}{r^2}}}   -  \frac{\cos{\left( \frac{\pi x}{2 r} \right)} - \sin{\left( \frac{\pi x}{2 r} \right)}\frac{\pi \delta x}{2 r}}{\sqrt{1 - \frac{x^2}{r^2}}} \frac{x\delta x}{r\sqrt{r^2 - x^2}}$$

Это значит, что при приращении $\delta x$, мы имеем приращение скалярного измерения 
$$Y_x(x + \delta x) - Y_x(x) = \frac{- \sin{\left( \frac{\pi x}{2 r} \right)}\frac{\pi \delta x}{2 r}}{\sqrt{1 - \frac{x^2}{r^2}}}   -  \frac{\cos{\left( \frac{\pi x}{2 r} \right)} - \sin{\left( \frac{\pi x}{2 r} \right)}\frac{\pi \delta x}{2 r}}{\sqrt{1 - \frac{x^2}{r^2}}} \frac{x\delta x}{r\sqrt{r^2 - x^2}}$$

А если мы оставим только линейные члены по $\delta x$, выйдет
$$Y_x(x + \delta x) - Y_x(x) = \frac{- \sin{\left( \frac{\pi x}{2 r} \right)}\frac{\pi \delta x}{2 r}}{\sqrt{1 - \frac{x^2}{r^2}}}   -  \frac{\cos{\left( \frac{\pi x}{2 r} \right)} }{\sqrt{1 - \frac{x^2}{r^2}}} \frac{x\delta x}{r\sqrt{r^2 - x^2}}$$

$$Y_x(x + \delta x) - Y_x(x) = -\frac{\delta x}{\sqrt{1 - \frac{x^2}{r^2}}}  \left( \sin{\left( \frac{\pi x}{2 r} \right)}\frac{\pi}{2 r} + \cos{\left( \frac{\pi x}{2 r} \right)}  \frac{x}{r\sqrt{r^2 - x^2}}   \right)$$

А дальше что? А дальше кот прошёлся по клавиатуре и у меня всё блин удалилось. Короче там в итоге $Y_x(x + \delta x) - Y_x(x) = 0$ только при $x=0$

###### **ВЫКЛАДКИ ДЛЯ КОРОТКОГО ДИПОЛЯ**

КА-1 в ССК-1 принимает сигнал от КА-2. В ССК-1:
- КА-1 в начале координат, $q=[1,0,0,0]$
- КА-2 в $r,q$ <br>
Если у каждого КА по 3 антенны, то в итоге 9 измерений

In [3]:
x1 = kf.local_dipole(None, r, 'x', model='short dipole').simplify()
y1 = kf.local_dipole(None, r, 'y', model='short dipole').simplify()
z1 = kf.local_dipole(None, r, 'z', model='short dipole').simplify()
x2 = kf.local_dipole(None, -kf.quart2dcm(q) @ r, 'x', model='short dipole').simplify()
y2 = kf.local_dipole(None, -kf.quart2dcm(q) @ r, 'y', model='short dipole').simplify()
z2 = kf.local_dipole(None, -kf.quart2dcm(q) @ r, 'z', model='short dipole').simplify()
kf.my_print('Усиления для КА-1', bold=True)
print('Усиление G вдоль оси x:')
display(x1)
print('Усиление G вдоль оси y:')
display(y1)
print('Усиление G вдоль оси z:')
display(z1)
kf.my_print('\nУсиления для КА-2', bold=True)
print('Усиление G вдоль оси x:')
display(x2)
print('Усиление G вдоль оси y:')
display(y2)
print('Усиление G вдоль оси z:')
display(z2)

Усиления для КА-1
Усиление G вдоль оси x:


1.0*sqrt((r_y**2 + r_z**2)/(r_x**2 + r_y**2 + r_z**2))

Усиление G вдоль оси y:


1.0*sqrt((r_x**2 + r_z**2)/(r_x**2 + r_y**2 + r_z**2))

Усиление G вдоль оси z:


...

1.0*sqrt(((2*r_x*(q_x*q_y - q_z*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) - r_y*(2*q_x**2 + 2*q_z**2 - 1) + 2*r_z*(q_x*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1) + q_y*q_z))**2 + (-2*r_x*(q_x*q_z + q_y*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) + 2*r_y*(q_x*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1) - q_y*q_z) + r_z*(2*q_x**2 + 2*q_y**2 - 1))**2)/((2*r_x*(q_x*q_y - q_z*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) - r_y*(2*q_x**2 + 2*q_z**2 - 1) + 2*r_z*(q_x*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1) + q_y*q_z))**2 + (-2*r_x*(q_x*q_z + q_y*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) + 2*r_y*(q_x*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1) - q_y*q_z) + r_z*(2*q_x**2 + 2*q_y**2 - 1))**2 + (-r_x*(2*q_y**2 + 2*q_z**2 - 1) + 2*r_y*(q_x*q_y + q_z*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) + 2*r_z*(q_x*q_z - q_y*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)))**2))

Усиление G вдоль оси y:


1.0*sqrt(((-2*r_x*(q_x*q_z + q_y*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) + 2*r_y*(q_x*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1) - q_y*q_z) + r_z*(2*q_x**2 + 2*q_y**2 - 1))**2 + (-r_x*(2*q_y**2 + 2*q_z**2 - 1) + 2*r_y*(q_x*q_y + q_z*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) + 2*r_z*(q_x*q_z - q_y*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)))**2)/((2*r_x*(q_x*q_y - q_z*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) - r_y*(2*q_x**2 + 2*q_z**2 - 1) + 2*r_z*(q_x*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1) + q_y*q_z))**2 + (-2*r_x*(q_x*q_z + q_y*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) + 2*r_y*(q_x*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1) - q_y*q_z) + r_z*(2*q_x**2 + 2*q_y**2 - 1))**2 + (-r_x*(2*q_y**2 + 2*q_z**2 - 1) + 2*r_y*(q_x*q_y + q_z*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) + 2*r_z*(q_x*q_z - q_y*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)))**2))

Усиление G вдоль оси z:


1.0*sqrt(((2*r_x*(q_x*q_y - q_z*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) - r_y*(2*q_x**2 + 2*q_z**2 - 1) + 2*r_z*(q_x*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1) + q_y*q_z))**2 + (-r_x*(2*q_y**2 + 2*q_z**2 - 1) + 2*r_y*(q_x*q_y + q_z*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) + 2*r_z*(q_x*q_z - q_y*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)))**2)/((2*r_x*(q_x*q_y - q_z*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) - r_y*(2*q_x**2 + 2*q_z**2 - 1) + 2*r_z*(q_x*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1) + q_y*q_z))**2 + (-2*r_x*(q_x*q_z + q_y*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) + 2*r_y*(q_x*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1) - q_y*q_z) + r_z*(2*q_x**2 + 2*q_y**2 - 1))**2 + (-r_x*(2*q_y**2 + 2*q_z**2 - 1) + 2*r_y*(q_x*q_y + q_z*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)) + 2*r_z*(q_x*q_z - q_y*sqrt(-q_x**2 - q_y**2 - q_z**2 + 1)))**2))

In [ ]:
h = kf.get_vars('h', 9)
n = sqrt(r[0]**2 + r[1]**2 + r[2]**2)
eqs = [Eq(h[0], n / sqrt(x1*x2)), Eq(h[1], n / sqrt(x1*y2)), Eq(h[2], n / sqrt(x1*z2)),
       Eq(h[3], n / sqrt(y1*x2)), Eq(h[4], n / sqrt(y1*y2)), Eq(h[5], n / sqrt(y1*z2)),
       Eq(h[6], n / sqrt(z1*x2)), Eq(h[7], n / sqrt(z1*y2)), Eq(h[8], n / sqrt(z1*z2))]
anw = solve(eqs, r)

**Не считается**

Теперь допустим, что КА-2 имеет 1 изотропнуб антенну

In [5]:
kf.my_print('G2=1, так что имеем готовые измерения:', bold=True)
h1 = (sqrt(r[0]**2 + r[1]**2 + r[2]**2) / sqrt(x1)).simplify()
h2 = (sqrt(r[0]**2 + r[1]**2 + r[2]**2) / sqrt(y1)).simplify()
h3 = (sqrt(r[0]**2 + r[1]**2 + r[2]**2) / sqrt(z1)).simplify()
print('h1:')
display(h1)
print('h2:')
display(h2)
print('h3:')
display(h3)

G2=1, так что имеем готовые измерения:
h1:


1.0*sqrt(r_x**2 + r_y**2 + r_z**2)/((r_y**2 + r_z**2)/(r_x**2 + r_y**2 + r_z**2))**(1/4)

h2:


1.0*sqrt(r_x**2 + r_y**2 + r_z**2)/((r_x**2 + r_z**2)/(r_x**2 + r_y**2 + r_z**2))**(1/4)

h3:


1.0*sqrt(r_x**2 + r_y**2 + r_z**2)/((r_x**2 + r_y**2)/(r_x**2 + r_y**2 + r_z**2))**(1/4)

In [42]:
tmp = (sqrt(sqrt(r.dot(r)**3 / (r[1]**2 + r[2]**2))))
print(f"Укороченная запись: h1 = ")
display(tmp)
print(f"Проверка: {float((h1 - tmp).subs([(r[0], 1), (r[1], 2), (r[2], 3)]))}")

Укороченная запись: h1 = 


((r_x**2 + r_y**2 + r_z**2)**3/(r_y**2 + r_z**2))**(1/4)

Проверка: 0.0


-2.417245056926546e-16

In [57]:
1/h1**4

1.0*(r_y**2 + r_z**2)/(r_x**2 + r_y**2 + r_z**2)**3

In [58]:
1/h2**4

1.0*(r_x**2 + r_z**2)/(r_x**2 + r_y**2 + r_z**2)**3

In [59]:
1/h3**4

1.0*(r_x**2 + r_y**2)/(r_x**2 + r_y**2 + r_z**2)**3

In [43]:
(2/(1/h1**4 + 1/h2**4 + 1/h3**4)).simplify()

1.0*(r_x**2 + r_y**2 + r_z**2)**2

$$Y_x(x,y,z) = \sqrt{1 - \frac{x^2}{x^2+y^2+z^2}}$$

$$Y_x(x + \delta x,y + \delta y,z + \delta z) = \sqrt{1 - \frac{x^2 + 2x\delta x}{x^2 +y^2 +z^2+ 2x\delta x+ 2y\delta y + 2z\delta z}} = \sqrt{1 - \frac{x^2 + 2x\delta x}{x^2 +y^2 +z^2}\frac{1}{1 + \frac{2x\delta x+ 2y\delta y + 2z\delta z}{x^2 +y^2 +z^2}}}$$

$$Y_x(x + \delta x,y + \delta y,z + \delta z) \approx 
\sqrt{1 - \frac{x^2 + 2x\delta x}{x^2 +y^2 +z^2}\left(1 - \frac{2x\delta x+ 2y\delta y + 2z\delta z}{x^2 +y^2 +z^2} \right)} =  
\sqrt{1 - \frac{x^2 + 2x\delta x}{x^2 +y^2 +z^2} + \frac{x^2 + 2x\delta x}{x^2 +y^2 +z^2} \frac{2x\delta x+ 2y\delta y + 2z\delta z}{x^2 +y^2 +z^2}}$$

###### Ненужные выкладки (удалить)

In [24]:
tmp = (x.subs(d_subs) - x).expand()
tmp

1.0*cos(pi*r_x/(2*sqrt(d**2)) + pi*δx/(2*sqrt(d**2)))/sqrt(r_y**2/d**2 + 2*r_y*δy/d**2 + r_z**2/d**2 + 2*r_z*δz/d**2 + δy**2/d**2 + δz**2/d**2) - 1.0*cos(pi*r_x/(2*sqrt(d**2)))/sqrt(r_y**2/d**2 + r_z**2/d**2)

In [10]:
d_subs = [(r[0], r[0] + dx), (r[1], r[1] + dy), (r[2], r[2] + dz)]
anw = sp.solve([sp.Eq(x.subs(d_subs),x), sp.Eq(y.subs(d_subs),y), sp.Eq(z.subs(d_subs),z)], [dx, dy, dz])
anw

[]

In [37]:
eqs = [sp.Eq(xr, x), sp.Eq(yr, y), sp.Eq(zr, z), sp.Eq(d**2, r.dot(r))]
anw = sp.solve(eqs, [d, r[0], r[1], r[2]])
anw

NotImplementedError: could not solve -x_r*sqrt((r_y**2 + r_z**2)/(r_x**2 + r_y**2 + r_z**2)) + cos(pi*r_x/(2*sqrt(r_x**2 + r_y**2 + r_z**2)))

##### **<u>Согласно статье</u> 038 (Shauying R.K.) Observability of Nonlinear Systems**

<span style="color:#2b817d">Примечание:</span>     $km \geq n$

<span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">055 (Yujiro Inowe) On the Observability of Autonomous Nonlinear Systems</span> <span style="color:#0ab49a">надо просто проверить ранг во всём $R^n$!</span>    $rg\frac{\partial \boldsymbol{H}_d}{\partial \boldsymbol{x}} (\boldsymbol{x}) = n \hskip20px \forall x \in R^n,$
$$d = n \frac{n +3}{2}.$$

<span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">051 (Andrew J. Whalen) Observability and Controllability of Nonlinear Networks The Role of Symmetry</span> <span style="color:#0ab49a">надо проверить</span> $\delta(x) = \frac{|\sigma_{min}[O^T O]|}{|\sigma_{max}[O^T O]|}.$

###### <u>Инициализация</u>

In [1]:
from common_func import *

o, num_params, t, ω, μ, ρ, r_orb, v_orb = init_symbol_params()

Инициализация проекта kiam-formation
Параметры не могут быть загружены! Нет файла: kiamformation/data/config_choose.csv
Матрицы Ф:(6, 6), Q:(3, 3), P:(6, 6), D:(6, 3)
Высота орбиты: 400 км
Период орбиты: 1.54 часов
Плотность атмосферы: 6.404751331738951e-12 кг/м³


###### <u>Алгоритм</u> (новое)

In [16]:
def my_diff(expr, power: int = 1, n_c=1, n_d=1):
    """
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной
    """    
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    global dr_d, dv_d, dq_d, dω_d, dr_c, dv_c, dq_c, dω_c
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for j in range(3):
        for i in range(n_d):
            subses += [(Derivative(r_d[i][j], t), dr_d[i][j])]
            subses += [(Derivative(v_d[i][j], t), dv_d[i][j])]
            subses += [(Derivative(q_d[i][j+1], t), dq_d[i][j+1])]
            subses += [(Derivative(ω_d[i][j], t), dω_d[i][j])]
        for i in range(n_c):
            subses += [(Derivative(r_c[i][j], t), dr_c[i][j])]
            subses += [(Derivative(v_c[i][j], t), dv_c[i][j])]
            subses += [(Derivative(q_c[i][j+1], t), dq_c[i][j+1])]
            subses += [(Derivative(ω_c[i][j], t), dω_c[i][j])]
    anw = anw.diff(t).subs(subses)
    anw.simplify()
    return anw

def SubRandParams(J, n_d: int, n_c: int, n_x: int, n_y: int, testprint: bool = False):
    """Берёт матрицу J размером n_x на n_y, подставляет случайные значения"""
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    # Генерация случайных параметров движения
    s_r = lambda: o.v.spread(param='r', name="FemtoSat")[0]
    s_v = lambda: o.v.spread(param='v', name="FemtoSat")[0]
    s_w = lambda: o.v.spread(param='w', name="FemtoSat")[0]
    rand_params = [(ω, num_params['ω0']), (pi, np.pi), (ρ, num_params['ρ']), (v_orb, num_params['v0']), 
                   (o.f.c_resist, num_params['Cd']), (o.c.c_resist, num_params['Cc']), 
                   (o.f.mass, num_params['md']), (o.c.mass, num_params['mc']),
                   (o.f.size[0], num_params['sd'][0]), (o.f.size[1], num_params['sd'][1]),
                   (o.c.size[0], num_params['sc'][0]), (o.c.size[1], num_params['sc'][1]),
                   (o.f.J[0, 0], num_params['Jd'][0, 0]), (o.f.J[1, 1], num_params['Jd'][1, 1]), (o.f.J[2, 2], num_params['Jd'][2, 2])] 
    for i in range(n_d):
        q = np.random.uniform(-1, 1, 4)
        for j in range(3):
            rand_params.extend([(r_d[i][j], s_r()), (v_d[i][j], s_v()), (ω_d[i][j], s_w()), (q_d[i][j+1], q[j+1] / np.linalg.norm(q))])
    for i in range(n_c):
        q = np.random.uniform(-1, 1, 4)
        for j in range(3):
            rand_params.extend([(r_c[i][j], s_r()), (v_c[i][j], s_v()), (ω_c[i][j], s_w()), (q_c[i][j+1], q[j+1] / np.linalg.norm(q))])

    # Якобиан матрицы наблюдаемости: J[измерение (H), состояние (X)] 
    J_numb = np.array([[float(J[i, j].subs(rand_params).subs(t,0)) for j in range(n_x)] for i in range(n_y)])
    return J_numb

def ShauyingObservabilitySufficientCondition(o, n_d: int, n_c: int, X: list, Y: list, testprint: bool = False, hand_written_deriv: int = None, is_only_xz: bool = False):
    """Проверка достаточного условия наблюдаемости системы. Проверка равномерного отношения миноров матрицы наблюдаемости.
    :param n_d: Количество чипсатов
    :param X: Список неизвестных параметров, которые необходимо найти
    :param Y: Список известных параметров (измерений системы в t₀=0)
    :param my_diff: Функция взятия производной по времени
    :param testprint: Флаг вывода экстра-информации"""
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    def print_and_record(report: str, lcl_txt: str):
        print(lcl_txt)
        return report + lcl_txt + "\n"
    report = kf.my_print(f"Количество кубсатов: {n_c}\nКоличество чипсатов: {n_d}\n", if_return=True, bold=True)

    # Количество одномоментных измерений
    l = len(Y)
    # Требуемое количество существующих производных функции измерения
    k = int(len(X) // len(Y)) if hand_written_deriv is None else hand_written_deriv
    txt = f"" if hand_written_deriv is None else f"\033[1mВнимание! Рассчитывается не отношение миноров, а ранг расширенного Якобиана\033[0m\n"
    report = print_and_record(report, txt + f"Неизвестные: n = {len(X)} (на каждый чипсат по {int(len(X) // n_d)} параметров)\nИзвестные: l = {l}\n∃ производные порядка k = {len(X) / len(Y)} (Должна быть целой!)")

    # Матрица наблюдаемости системы
    if is_only_xz:
        f = Matrix([Matrix([*dr_d[i][::2], *dv_d[i][::2]]) for i in range(len(dr_d))])
    else:
        f = Matrix([Matrix([*dr_d[i], *dq_d[i][1:4], *dv_d[i], *dω_d[i]]) for i in range(len(dr_d))]) if o.v.NAVIGATION_ANGLES else Matrix([Matrix([*dr_d[i], *dv_d[i]]) for i in range(len(dr_d))])
    H1 = Y
    H = []
    for kk in range(k):
        if kk > 0:
            dhdx = Matrix([[H1[ll].diff(X[j]) for j in range(len(X))] for ll in range(l)])
            dhdx_f = dhdx @ f
            dhdx_f.simplify()
        for ll in range(l):
            if testprint:
                print(f"__расчёт матрицы H_: k={(kk+1)}/{k}, l={(ll+1)}/{l}")
            H1[ll] = Y[ll] if kk == 0 else my_diff(H1[ll], n_c=n_c, n_d=n_d) + dhdx_f[ll]
        H.extend(H1)
    H = Matrix(H)
    report = print_and_record(report, f"Размерность матрицы H: {shape(H)}")

    # Якобиан матрицы наблюдаемости: J[измерение (H), состояние (X)]
    J = Matrix([[H[i].diff(X[j]) for j in range(len(X))] for i in range(k * l)])
    report = print_and_record(report, f"Размерность матрицы J: {shape(J)}")

    # Подстановка конкретных значений
    J_numb = SubRandParams(J=J, n_c=n_c, n_d=n_d, n_x=len(X), n_y=k*l, testprint=testprint)
    _, v, _ = np.linalg.svd(J_numb.T @ J_numb)
    report = print_and_record(report, f"v = {v}")
    report = print_and_record(report, f"σₘₙ/σₘₐₓ = {np.min(v)}/{np.max(v)} = {np.min(v) / np.max(v)} | σ>10⁻⁷: {np.sum(v>1e-7)}/{len(v)} (f-051)")

    # Достаточное условие
    txt = f"\nРанг матрицы: {[np.linalg.matrix_rank(J_numb, tol=tol) for tol in [1e-3, 1e-5, 1e-7, 1e-10, 1e-12, 1e-15]]} (f-055)\n"
    txt += f"Детерминант матрицы: {np.linalg.det(J_numb)}\n" if J_numb.shape[0] == J_numb.shape[1] else ""
    if hand_written_deriv is None:
        report = print_and_record(report, txt + f"Следующие параметры не должны быть нулевыми:\n")
        d, Δ, flag, i_min = [], [], True, -1
        for i in range(len(X)):
            tmp = kf.principal_minor(J_numb, i)
            d += [tmp if i == 0 else tmp / Δ[-1]]
            Δ += [tmp]
            report = print_and_record(report, f"Δ_{i} = {d[-1]}" if i == 0 else f"Δ_{i} / Δ_{i-1} = {d[-1]}")
        
            # Чек наблюдаемости
            if flag and abs(d[-1]) < 1e-10:
                i_min = i
                flag = False
            if not flag:
                break
    
        # Вывод
        if flag:
            txt = f"\n\033[1mВыполнено достаточное условие! Система наблюдаема\033[0m"
        else:
            in_txt = f"Δ_{i_min}" if i_min == 0 else f"Δ_{i_min} / Δ_{i_min-1}"
            txt = f"\n\033[1mНе выполнено достаточное условие. Нулевой параметр: {in_txt} = {d[i_min]}\033[0m"
        report = print_and_record(report, txt)
        return H, J, J_numb, Δ, report
    report = print_and_record(report, txt)
    return H, J, J_numb, None, report
    

def observe_system(n_c, n_d, is_d_qw, gains, is_drag, is_only_xz, testprint, hand_written_deriv, disp: bool = True, Hcheck: bool = True):
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c, t
    global dr_d, dv_d, dq_d, dω_d, dr_c, dv_c, dq_c, dω_c, dydq
    r_d, v_d, q_d, ω_d = get_state_vector(func=kf.get_func, obj='d', n=n_d)
    r_c, v_c, q_c, ω_c = get_state_vector(func=kf.get_func, obj='c', n=n_c)

    o.c.n = n_c
    o.f.n = n_d
    o.f.r_orf, o.f.v_orf, o.f.q, o.f.w_brf = r_d, v_d, q_d, ω_d
    o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_brf = r_c, v_c, q_c, ω_c

    x = []
    for i in range(n_d):
        x.extend(r_d[i])
        if is_d_qw:
            x.extend(q_d[i][1:4])
        x.extend(v_d[i])
        if is_d_qw:
            x.extend(ω_d[i])
    x = Matrix(x)
    
    kf.my_print(f"Вектор состояния:", bold=True)
    display(x.T)

    o.v.GAIN_MODEL_C_N = gains['cN']
    o.v.GAIN_MODEL_F_N = gains['dN']
    o.v.init_choice_params()
    o.c.gain_mode = o.v.GAIN_MODEL_C = gains['cModel']
    o.f.gain_mode = o.v.GAIN_MODEL_F = gains['dModel']
    o.v.NAVIGATION_ANGLES = is_d_qw
    o.v.DYNAMIC_MODEL['aero drag'] = is_drag
    o.v.MULTI_ANTENNA_TAKE = True
    o.v.MULTI_ANTENNA_SEND = True
    
    y, dydq, notes = kf.measure_antennas_power(c=o.c, f=o.f, v=o.v, p=o.p, j=int(len(x) // n_d), estimated_params=x, t=t)
    if disp:
        kf.my_print(f"Вектор измерений:", bold=True)
        display(y)

    # >>>>>>>>>>>>>>>>> Проверка матрицы H <<<<<<<<<<<<<<<<<
    if Hcheck:
        H_1 = kf.h_matrix(t=t, v=o.v, f=o.f, c=o.c, r_f=r_d, r_c=r_c, 
                          q_f=[Matrix(q_d[i][1:4]) for i in range(o.f.n)], q_c=[Matrix(q_c[i][1:4]) for i in range(o.c.n)])
        H_2 = zeros(*H_1.shape)
        for ix, xx in enumerate(x):
            for iy, yy in enumerate(y):
                H_2[iy, ix] = yy.diff(xx)
        kf.my_print("Проверка матрицы H:", bold=True)
        tmp = H_1 - H_2
        tmp.simplify()
        display(tmp)

    if is_only_xz:
        tmp = []
        for i in range(n_d):
            tmp.extend([r_d[i][0], r_d[i][2], v_d[i][0], v_d[i][2]])
        x = Matrix(tmp)
        kf.my_print(f"Вектор состояния (изменённый):", bold=True)
        display(x.T)
    
    dr_d, dv_d, dq_d, dω_d = ([0 for _ in range(n_d)] for _ in range(4))
    dr_c, dv_c, dq_c, dω_c = ([0 for _ in range(n_c)] for _ in range(4))
    for i in range(n_d):
        dr_d[i], dv_d[i] = kf.translate_rhs(vrs=o.v, obj=o.f, i=i, rv=(r_d[i], v_d[i]), w=ω, mu=μ, rho=ρ)
        dq_d[i], dω_d[i] = kf.attitude_rhs(v=o.v, obj=o.f, t=t, i=i, qw=(q_d[i], ω_d[i]))
    for i in range(n_c):
        dr_c[i], dv_c[i] = kf.translate_rhs(vrs=o.v, obj=o.c, i=i, rv=(r_c[i], v_c[i]), w=ω, mu=μ, rho=ρ)
        dq_c[i], dω_c[i] = kf.attitude_rhs(v=o.v, obj=o.c, t=t, i=i, qw=(q_c[i], ω_c[i]))
    if disp:
        kf.my_print(f"Динамика: \n{' '*10}dr = ", bold=True)
        display(dr_d[0])
        kf.my_print(f"{' '*10}dv = ", bold=True)
        display(dv_d[0])
        kf.my_print(f"{' '*10}dq = ", bold=True)
        display(dq_d[0])
        kf.my_print(f"{' '*10}dω = ", bold=True)
    
    return ShauyingObservabilitySufficientCondition(o=o, testprint=True, n_c=n_c, n_d=n_d, X=x, Y=y, hand_written_deriv=hand_written_deriv, is_only_xz=is_only_xz)

###### <u>Наблюдаемость системы</u> без аэродинамики и углового движения, антенны изотропные

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=2, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=True, Hcheck=True)

In [3]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=5, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=True, Hcheck=True)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t), r_1^d_x(t), r_1^d_y(t), r_1^d_z(t), v_1^d_x(t), v_1^d_y(t), v_1^d_z(t), r_2^d_x(t), r_2^d_y(t), r_2^d_z(t), v_2^d_x(t), v_2^d_y(t), v_2^d_z(t), r_3^d_x(t), r_3^d_y(t), r_3^d_z(t), v_3^d_x(t), v_3^d_y(t), v_3^d_z(t), r_4^d_x(t), r_4^d_y(t), r_4^d_z(t), v_4^d_x(t), v_4^d_y(t), v_4^d_z(t)]])

Вектор измерений:


Matrix([
[   1/((r_0^c_x(t) - r_0^d_x(t))**2 + (r_0^c_y(t) - r_0^d_y(t))**2 + (r_0^c_z(t) - r_0^d_z(t))**2)],
[   1/((r_0^c_x(t) - r_1^d_x(t))**2 + (r_0^c_y(t) - r_1^d_y(t))**2 + (r_0^c_z(t) - r_1^d_z(t))**2)],
[   1/((r_0^c_x(t) - r_2^d_x(t))**2 + (r_0^c_y(t) - r_2^d_y(t))**2 + (r_0^c_z(t) - r_2^d_z(t))**2)],
[   1/((r_0^c_x(t) - r_3^d_x(t))**2 + (r_0^c_y(t) - r_3^d_y(t))**2 + (r_0^c_z(t) - r_3^d_z(t))**2)],
[   1/((r_0^c_x(t) - r_4^d_x(t))**2 + (r_0^c_y(t) - r_4^d_y(t))**2 + (r_0^c_z(t) - r_4^d_z(t))**2)],
[1/((-r_0^d_x(t) + r_1^d_x(t))**2 + (-r_0^d_y(t) + r_1^d_y(t))**2 + (-r_0^d_z(t) + r_1^d_z(t))**2)],
[1/((-r_0^d_x(t) + r_2^d_x(t))**2 + (-r_0^d_y(t) + r_2^d_y(t))**2 + (-r_0^d_z(t) + r_2^d_z(t))**2)],
[1/((-r_1^d_x(t) + r_2^d_x(t))**2 + (-r_1^d_y(t) + r_2^d_y(t))**2 + (-r_1^d_z(t) + r_2^d_z(t))**2)],
[1/((-r_0^d_x(t) + r_3^d_x(t))**2 + (-r_0^d_y(t) + r_3^d_y(t))**2 + (-r_0^d_z(t) + r_3^d_z(t))**2)],
[1/((-r_1^d_x(t) + r_3^d_x(t))**2 + (-r_1^d_y(t) + r_3^d_y(t))**2 + (-r_1^d_z(t) +

Проверка матрицы H:


...

          dv = 


Matrix([
[                     -2*w_0*v_0^d_z(t)],
[                    -w_0**2*r_0^d_y(t)],
[3*w_0**2*r_0^d_z(t) + 2*w_0*v_0^d_x(t)]])

          dq = 


Matrix([
[                                             -0.5*q_0^d_x(t)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_y(t) - 0.5*q_0^d_z(t)*ω_0^d_z(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_z(t) + 0.5*q_0^d_z(t)*ω_0^d_y(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_y(t) + 0.5*q_0^d_x(t)*ω_0^d_z(t) - 0.5*q_0^d_z(t)*ω_0^d_x(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_z(t) - 0.5*q_0^d_x(t)*ω_0^d_y(t) + 0.5*q_0^d_y(t)*ω_0^d_x(t)]])

          dω = 
Количество кубсатов: 1
Количество чипсатов: 5

Неизвестные: n = 30 (на каждый чипсат по 6 параметров)
Известные: l = 15
∃ производные порядка k = 2.0 (Должна быть целой!)
Критерий (055): происзводные порядка 33.0
a: None
a: None
_расчёт матрицы H_: k=1/2, l=1/15
_расчёт матрицы H_: k=1/2, l=2/15
_расчёт матрицы H_: k=1/2, l=3/15
_расчёт матрицы H_: k=1/2, l=4/15
_расчёт матрицы H_: k=1/2, l=5/15
_расчёт матрицы H_: k=1/2, l=6/15
_расчёт матрицы H_: k=1/2, l=7/15
_расчёт матрицы H_: k=1/2, l=8/15
_расчёт матрицы H_: k=1/2, l=9/15
_расчёт матрицы H_: k=1/2, l=10/15
_расчёт матрицы H_: k=1/2, l=11/15
_расчёт матрицы H_: k=1/2, l=12/15
_расчёт матрицы H_: k=1/2, l=13/15
_расчёт матрицы H_: k=1/2, l=14/15
_расчёт матрицы H_: k=1/2, l=15/15
a: None
a: None
_расчёт матрицы H_: k=2/2, l=1/15
_расчёт матрицы H_: k=2/2, l=2/15
_расчёт матрицы H_: k=2/2, l=3/15
_расчёт матрицы H_: k=2/2, l=4/15
_расчёт матрицы H_: k=2/2, l=5/15
_расчёт матрицы H_: k=2/2, l=6/15
_расчёт матрицы H_:

In [18]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=11, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t), r_1^d_x(t), r_1^d_y(t), r_1^d_z(t), v_1^d_x(t), v_1^d_y(t), v_1^d_z(t), r_2^d_x(t), r_2^d_y(t), r_2^d_z(t), v_2^d_x(t), v_2^d_y(t), v_2^d_z(t), r_3^d_x(t), r_3^d_y(t), r_3^d_z(t), v_3^d_x(t), v_3^d_y(t), v_3^d_z(t), r_4^d_x(t), r_4^d_y(t), r_4^d_z(t), v_4^d_x(t), v_4^d_y(t), v_4^d_z(t), r_5^d_x(t), r_5^d_y(t), r_5^d_z(t), v_5^d_x(t), v_5^d_y(t), v_5^d_z(t), r_6^d_x(t), r_6^d_y(t), r_6^d_z(t), v_6^d_x(t), v_6^d_y(t), v_6^d_z(t), r_7^d_x(t), r_7^d_y(t), r_7^d_z(t), v_7^d_x(t), v_7^d_y(t), v_7^d_z(t), r_8^d_x(t), r_8^d_y(t), r_8^d_z(t), v_8^d_x(t), v_8^d_y(t), v_8^d_z(t), r_9^d_x(t), r_9^d_y(t), r_9^d_z(t), v_9^d_x(t), v_9^d_y(t), v_9^d_z(t), r_10^d_x(t), r_10^d_y(t), r_10^d_z(t), v_10^d_x(t), v_10^d_y(t), v_10^d_z(t)]])

Количество кубсатов: 1
Количество чипсатов: 11

Неизвестные: n = 66 (на каждый чипсат по 6 параметров)
Известные: l = 66
∃ производные порядка k = 1.0 (Должна быть целой!)
Критерий (055): происзводные порядка 34.5
_расчёт матрицы H_: k=1/1, l=1/66
_расчёт матрицы H_: k=1/1, l=2/66
_расчёт матрицы H_: k=1/1, l=3/66
_расчёт матрицы H_: k=1/1, l=4/66
_расчёт матрицы H_: k=1/1, l=5/66
_расчёт матрицы H_: k=1/1, l=6/66
_расчёт матрицы H_: k=1/1, l=7/66
_расчёт матрицы H_: k=1/1, l=8/66
_расчёт матрицы H_: k=1/1, l=9/66
_расчёт матрицы H_: k=1/1, l=10/66
_расчёт матрицы H_: k=1/1, l=11/66
_расчёт матрицы H_: k=1/1, l=12/66
_расчёт матрицы H_: k=1/1, l=13/66
_расчёт матрицы H_: k=1/1, l=14/66
_расчёт матрицы H_: k=1/1, l=15/66
_расчёт матрицы H_: k=1/1, l=16/66
_расчёт матрицы H_: k=1/1, l=17/66
_расчёт матрицы H_: k=1/1, l=18/66
_расчёт матрицы H_: k=1/1, l=19/66
_расчёт матрицы H_: k=1/1, l=20/66
_расчёт матрицы H_: k=1/1, l=21/66
_расчёт матрицы H_: k=1/1, l=22/66
_расчёт матрицы H_: k=1/1

In [4]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=True, Hcheck=True)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор измерений:


Matrix([[1/((r_0^c_x(t) - r_0^d_x(t))**2 + (r_0^c_y(t) - r_0^d_y(t))**2 + (r_0^c_z(t) - r_0^d_z(t))**2)]])

Проверка матрицы H:


...

          dv = 


Matrix([
[                     -2*w_0*v_0^d_z(t)],
[                    -w_0**2*r_0^d_y(t)],
[3*w_0**2*r_0^d_z(t) + 2*w_0*v_0^d_x(t)]])

          dq = 


Matrix([
[                                             -0.5*q_0^d_x(t)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_y(t) - 0.5*q_0^d_z(t)*ω_0^d_z(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_z(t) + 0.5*q_0^d_z(t)*ω_0^d_y(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_y(t) + 0.5*q_0^d_x(t)*ω_0^d_z(t) - 0.5*q_0^d_z(t)*ω_0^d_x(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_z(t) - 0.5*q_0^d_x(t)*ω_0^d_y(t) + 0.5*q_0^d_y(t)*ω_0^d_x(t)]])

          dω = 
Количество кубсатов: 1
Количество чипсатов: 1

Неизвестные: n = 4 (на каждый чипсат по 4 параметров)
Известные: l = 1
∃ производные порядка k = 4.0 (Должна быть целой!)
Критерий (055): происзводные порядка 14.0
a: None
a: None
_расчёт матрицы H_: k=1/4, l=1/1
a: None
a: None
_расчёт матрицы H_: k=2/4, l=1/1
a: None
a: None
_расчёт матрицы H_: k=3/4, l=1/1
a: None
a: None
_расчёт матрицы H_: k=4/4, l=1/1
Размерность матрицы H: (4, 1)
Размерность матрицы J: (4, 4)
v = [1.27096596e-11 3.17740447e-12 2.57266722e-20 9.45472562e-24]
σₘₙ/σₘₐₓ = 9.45472562060678e-24/1.270965956067551e-11 = 7.43900776843803e-13 | σ>10⁻⁵: 0/4 (статья 051)

Ранг матрицы: [0, 0, 2, 3, 4, 4] (статья 055)
Детерминант матрицы: -3.1341709751969656e-33
Следующие параметры не должны быть нулевыми:

Δ_0 = -3.37262056207856e-27

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = -3.37262056207856e-27


In [20]:
H, J, Jn, Δ, report = observe_system(n_c=2, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=True, Hcheck=True)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор измерений:


Matrix([
[   1/((r_0^c_x(t) - r_0^d_x(t))**2 + (r_0^c_y(t) - r_0^d_y(t))**2 + (r_0^c_z(t) - r_0^d_z(t))**2)],
[1/((-r_0^d_x(t) + r_1^c_x(t))**2 + (-r_0^d_y(t) + r_1^c_y(t))**2 + (-r_0^d_z(t) + r_1^c_z(t))**2)]])

Проверка матрицы H:


...

          dv = 


Matrix([
[                     -2*w_0*v_0^d_z(t)],
[                    -w_0**2*r_0^d_y(t)],
[3*w_0**2*r_0^d_z(t) + 2*w_0*v_0^d_x(t)]])

          dq = 


Matrix([
[                                             -0.5*q_0^d_x(t)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_y(t) - 0.5*q_0^d_z(t)*ω_0^d_z(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_z(t) + 0.5*q_0^d_z(t)*ω_0^d_y(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_y(t) + 0.5*q_0^d_x(t)*ω_0^d_z(t) - 0.5*q_0^d_z(t)*ω_0^d_x(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_z(t) - 0.5*q_0^d_x(t)*ω_0^d_y(t) + 0.5*q_0^d_y(t)*ω_0^d_x(t)]])

          dω = 
Количество кубсатов: 2
Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 2
∃ производные порядка k = 3.0 (Должна быть целой!)
Критерий (055): происзводные порядка 13.5
_расчёт матрицы H_: k=1/3, l=1/2
_расчёт матрицы H_: k=1/3, l=2/2
_расчёт матрицы H_: k=2/3, l=1/2
_расчёт матрицы H_: k=2/3, l=2/2
_расчёт матрицы H_: k=3/3, l=1/2
_расчёт матрицы H_: k=3/3, l=2/2
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)
v = [2.39379046e-12 5.98446604e-13 1.20814341e-14 3.01849730e-15
 1.54941368e-17 2.41932482e-24]
σₘₙ/σₘₐₓ = 2.4193248219138535e-24/2.393790455763321e-12 = 1.0106669178536725e-12 | σ>10⁻⁵: 0/6 (статья 051)

Ранг матрицы: [0, 0, 3, 5, 6, 6] (статья 055)
Детерминант матрицы: -4.425292639295384e-47
Следующие параметры не должны быть нулевыми:

Δ_0 = 7.718848783651596e-41

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 7.718848783651596e-41


In [21]:
H, J, Jn, Δ, report = observe_system(n_c=2, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=True, Hcheck=True)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор измерений:


Matrix([
[   1/((r_0^c_x(t) - r_0^d_x(t))**2 + (r_0^c_y(t) - r_0^d_y(t))**2 + (r_0^c_z(t) - r_0^d_z(t))**2)],
[1/((-r_0^d_x(t) + r_1^c_x(t))**2 + (-r_0^d_y(t) + r_1^c_y(t))**2 + (-r_0^d_z(t) + r_1^c_z(t))**2)]])

Проверка матрицы H:


...

          dv = 


Matrix([
[                     -2*w_0*v_0^d_z(t)],
[                    -w_0**2*r_0^d_y(t)],
[3*w_0**2*r_0^d_z(t) + 2*w_0*v_0^d_x(t)]])

          dq = 


Matrix([
[                                             -0.5*q_0^d_x(t)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_y(t) - 0.5*q_0^d_z(t)*ω_0^d_z(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_z(t) + 0.5*q_0^d_z(t)*ω_0^d_y(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_y(t) + 0.5*q_0^d_x(t)*ω_0^d_z(t) - 0.5*q_0^d_z(t)*ω_0^d_x(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_z(t) - 0.5*q_0^d_x(t)*ω_0^d_y(t) + 0.5*q_0^d_y(t)*ω_0^d_x(t)]])

          dω = 
Количество кубсатов: 2
Количество чипсатов: 1

Неизвестные: n = 4 (на каждый чипсат по 4 параметров)
Известные: l = 2
∃ производные порядка k = 2.0 (Должна быть целой!)
Критерий (055): происзводные порядка 7.0
_расчёт матрицы H_: k=1/2, l=1/2
_расчёт матрицы H_: k=1/2, l=2/2
_расчёт матрицы H_: k=2/2, l=1/2
_расчёт матрицы H_: k=2/2, l=2/2
Размерность матрицы H: (4, 1)
Размерность матрицы J: (4, 4)
v = [5.64505521e-12 1.41125534e-12 1.00240343e-14 2.50600588e-15]
σₘₙ/σₘₐₓ = 2.5060058806704693e-15/5.645055205529411e-12 = 0.0004439293841122761 | σ>10⁻⁵: 0/4 (статья 051)

Ранг матрицы: [0, 0, 3, 4, 4, 4] (статья 055)
Детерминант матрицы: 1.4146506758657842e-26
Следующие параметры не должны быть нулевыми:

Δ_0 = -3.409860090481984e-20

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = -3.409860090481984e-20


In [22]:
H, J, Jn, Δ, report = observe_system(n_c=3, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=True, Hcheck=True)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор измерений:


Matrix([
[   1/((r_0^c_x(t) - r_0^d_x(t))**2 + (r_0^c_y(t) - r_0^d_y(t))**2 + (r_0^c_z(t) - r_0^d_z(t))**2)],
[1/((-r_0^d_x(t) + r_1^c_x(t))**2 + (-r_0^d_y(t) + r_1^c_y(t))**2 + (-r_0^d_z(t) + r_1^c_z(t))**2)],
[1/((-r_0^d_x(t) + r_2^c_x(t))**2 + (-r_0^d_y(t) + r_2^c_y(t))**2 + (-r_0^d_z(t) + r_2^c_z(t))**2)]])

Проверка матрицы H:


...

          dv = 


Matrix([
[                     -2*w_0*v_0^d_z(t)],
[                    -w_0**2*r_0^d_y(t)],
[3*w_0**2*r_0^d_z(t) + 2*w_0*v_0^d_x(t)]])

          dq = 


Matrix([
[                                             -0.5*q_0^d_x(t)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_y(t) - 0.5*q_0^d_z(t)*ω_0^d_z(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_z(t) + 0.5*q_0^d_z(t)*ω_0^d_y(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_y(t) + 0.5*q_0^d_x(t)*ω_0^d_z(t) - 0.5*q_0^d_z(t)*ω_0^d_x(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_z(t) - 0.5*q_0^d_x(t)*ω_0^d_y(t) + 0.5*q_0^d_y(t)*ω_0^d_x(t)]])

          dω = 
Количество кубсатов: 3
Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 3
∃ производные порядка k = 2.0 (Должна быть целой!)
Критерий (055): происзводные порядка 9.0
_расчёт матрицы H_: k=1/2, l=1/3
_расчёт матрицы H_: k=1/2, l=2/3
_расчёт матрицы H_: k=1/2, l=3/3
_расчёт матрицы H_: k=2/2, l=1/3
_расчёт матрицы H_: k=2/2, l=2/3
_расчёт матрицы H_: k=2/2, l=3/3
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)
v = [3.30602524e-11 1.34800053e-11 8.26502323e-12 3.36995622e-12
 2.35065775e-14 5.86949778e-15]
σₘₙ/σₘₐₓ = 5.869497775830564e-15/3.306025239349316e-11 = 0.00017753941216086373 | σ>10⁻⁵: 0/6 (статья 051)

Ранг матрицы: [0, 0, 5, 6, 6, 6] (статья 055)
Детерминант матрицы: 1.3086618856841873e-36
Следующие параметры не должны быть нулевыми:

Δ_0 = 1.401818648470408e-29

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 1.401818648470408e-29


In [23]:
H, J, Jn, Δ, report = observe_system(n_c=6, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=True, Hcheck=True)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор измерений:


Matrix([
[   1/((r_0^c_x(t) - r_0^d_x(t))**2 + (r_0^c_y(t) - r_0^d_y(t))**2 + (r_0^c_z(t) - r_0^d_z(t))**2)],
[1/((-r_0^d_x(t) + r_1^c_x(t))**2 + (-r_0^d_y(t) + r_1^c_y(t))**2 + (-r_0^d_z(t) + r_1^c_z(t))**2)],
[1/((-r_0^d_x(t) + r_2^c_x(t))**2 + (-r_0^d_y(t) + r_2^c_y(t))**2 + (-r_0^d_z(t) + r_2^c_z(t))**2)],
[1/((-r_0^d_x(t) + r_3^c_x(t))**2 + (-r_0^d_y(t) + r_3^c_y(t))**2 + (-r_0^d_z(t) + r_3^c_z(t))**2)],
[1/((-r_0^d_x(t) + r_4^c_x(t))**2 + (-r_0^d_y(t) + r_4^c_y(t))**2 + (-r_0^d_z(t) + r_4^c_z(t))**2)],
[1/((-r_0^d_x(t) + r_5^c_x(t))**2 + (-r_0^d_y(t) + r_5^c_y(t))**2 + (-r_0^d_z(t) + r_5^c_z(t))**2)]])

Проверка матрицы H:


...

          dv = 


Matrix([
[                     -2*w_0*v_0^d_z(t)],
[                    -w_0**2*r_0^d_y(t)],
[3*w_0**2*r_0^d_z(t) + 2*w_0*v_0^d_x(t)]])

          dq = 


Matrix([
[                                             -0.5*q_0^d_x(t)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_y(t) - 0.5*q_0^d_z(t)*ω_0^d_z(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_z(t) + 0.5*q_0^d_z(t)*ω_0^d_y(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_y(t) + 0.5*q_0^d_x(t)*ω_0^d_z(t) - 0.5*q_0^d_z(t)*ω_0^d_x(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_z(t) - 0.5*q_0^d_x(t)*ω_0^d_y(t) + 0.5*q_0^d_y(t)*ω_0^d_x(t)]])

          dω = 
Количество кубсатов: 6
Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 6
∃ производные порядка k = 1.0 (Должна быть целой!)
Критерий (055): происзводные порядка 4.5
_расчёт матрицы H_: k=1/1, l=1/6
_расчёт матрицы H_: k=1/1, l=2/6
_расчёт матрицы H_: k=1/1, l=3/6
_расчёт матрицы H_: k=1/1, l=4/6
_расчёт матрицы H_: k=1/1, l=5/6
_расчёт матрицы H_: k=1/1, l=6/6
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)
v = [7.67350704e-10 6.64408111e-13 5.59258664e-15 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
σₘₙ/σₘₐₓ = 0.0/7.673507040489054e-10 = 0.0 | σ>10⁻⁵: 0/6 (статья 051)

Ранг матрицы: [0, 1, 2, 3, 3, 3] (статья 055)
Детерминант матрицы: 0.0
Следующие параметры не должны быть нулевыми:

Δ_0 = 0.0

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 0.0


###### <u>Наблюдаемость системы</u> c аэродинамикой, без углового движения, антенны изотропные

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=2, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=True, Hcheck=True)

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=True, Hcheck=True)

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=2, 
                                     n_d=5, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=True, Hcheck=True)

###### <u>Наблюдаемость системы</u> без аэродинамики, без углового движения, 1 антенна - 1/4 монополь

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=5, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 1, 'cModel': '1 antenna', 'dN': 1, 'dModel': '1 antenna'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

In [16]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=11, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 1, 'cModel': '1 antenna', 'dN': 1, 'dModel': '1 antenna'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t), r_1^d_x(t), r_1^d_y(t), r_1^d_z(t), v_1^d_x(t), v_1^d_y(t), v_1^d_z(t), r_2^d_x(t), r_2^d_y(t), r_2^d_z(t), v_2^d_x(t), v_2^d_y(t), v_2^d_z(t), r_3^d_x(t), r_3^d_y(t), r_3^d_z(t), v_3^d_x(t), v_3^d_y(t), v_3^d_z(t), r_4^d_x(t), r_4^d_y(t), r_4^d_z(t), v_4^d_x(t), v_4^d_y(t), v_4^d_z(t), r_5^d_x(t), r_5^d_y(t), r_5^d_z(t), v_5^d_x(t), v_5^d_y(t), v_5^d_z(t), r_6^d_x(t), r_6^d_y(t), r_6^d_z(t), v_6^d_x(t), v_6^d_y(t), v_6^d_z(t), r_7^d_x(t), r_7^d_y(t), r_7^d_z(t), v_7^d_x(t), v_7^d_y(t), v_7^d_z(t), r_8^d_x(t), r_8^d_y(t), r_8^d_z(t), v_8^d_x(t), v_8^d_y(t), v_8^d_z(t), r_9^d_x(t), r_9^d_y(t), r_9^d_z(t), v_9^d_x(t), v_9^d_y(t), v_9^d_z(t), r_10^d_x(t), r_10^d_y(t), r_10^d_z(t), v_10^d_x(t), v_10^d_y(t), v_10^d_z(t)]])

Количество кубсатов: 1
Количество чипсатов: 11

Неизвестные: n = 66 (на каждый чипсат по 6 параметров)
Известные: l = 66
∃ производные порядка k = 1.0 (Должна быть целой!)
Критерий (055): происзводные порядка 34.5
_расчёт матрицы H_: k=1/1, l=1/66
_расчёт матрицы H_: k=1/1, l=2/66
_расчёт матрицы H_: k=1/1, l=3/66
_расчёт матрицы H_: k=1/1, l=4/66
_расчёт матрицы H_: k=1/1, l=5/66
_расчёт матрицы H_: k=1/1, l=6/66
_расчёт матрицы H_: k=1/1, l=7/66
_расчёт матрицы H_: k=1/1, l=8/66
_расчёт матрицы H_: k=1/1, l=9/66
_расчёт матрицы H_: k=1/1, l=10/66
_расчёт матрицы H_: k=1/1, l=11/66
_расчёт матрицы H_: k=1/1, l=12/66
_расчёт матрицы H_: k=1/1, l=13/66
_расчёт матрицы H_: k=1/1, l=14/66
_расчёт матрицы H_: k=1/1, l=15/66
_расчёт матрицы H_: k=1/1, l=16/66
_расчёт матрицы H_: k=1/1, l=17/66
_расчёт матрицы H_: k=1/1, l=18/66
_расчёт матрицы H_: k=1/1, l=19/66
_расчёт матрицы H_: k=1/1, l=20/66
_расчёт матрицы H_: k=1/1, l=21/66
_расчёт матрицы H_: k=1/1, l=22/66
_расчёт матрицы H_: k=1/1


KeyboardInterrupt



In [18]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=7, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 1, 'cModel': '1 antenna', 'dN': 1, 'dModel': '1 antenna'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t), r_1^d_x(t), r_1^d_y(t), r_1^d_z(t), v_1^d_x(t), v_1^d_y(t), v_1^d_z(t), r_2^d_x(t), r_2^d_y(t), r_2^d_z(t), v_2^d_x(t), v_2^d_y(t), v_2^d_z(t), r_3^d_x(t), r_3^d_y(t), r_3^d_z(t), v_3^d_x(t), v_3^d_y(t), v_3^d_z(t), r_4^d_x(t), r_4^d_y(t), r_4^d_z(t), v_4^d_x(t), v_4^d_y(t), v_4^d_z(t), r_5^d_x(t), r_5^d_y(t), r_5^d_z(t), v_5^d_x(t), v_5^d_y(t), v_5^d_z(t), r_6^d_x(t), r_6^d_y(t), r_6^d_z(t), v_6^d_x(t), v_6^d_y(t), v_6^d_z(t)]])

Вектор состояния (изменённый):


Matrix([
[r_0^d_x(t), r_1^d_x(t), r_2^d_x(t), r_3^d_x(t), r_4^d_x(t), r_5^d_x(t), r_6^d_x(t)],
[r_0^d_z(t), r_1^d_z(t), r_2^d_z(t), r_3^d_z(t), r_4^d_z(t), r_5^d_z(t), r_6^d_z(t)],
[v_0^d_x(t), v_1^d_x(t), v_2^d_x(t), v_3^d_x(t), v_4^d_x(t), v_5^d_x(t), v_6^d_x(t)],
[v_0^d_z(t), v_1^d_z(t), v_2^d_z(t), v_3^d_z(t), v_4^d_z(t), v_5^d_z(t), v_6^d_z(t)]])

Количество кубсатов: 1
Количество чипсатов: 7

Неизвестные: n = 28 (на каждый чипсат по 4 параметров)
Известные: l = 28
∃ производные порядка k = 1.0 (Должна быть целой!)
Критерий (055): происзводные порядка 15.5
_расчёт матрицы H_: k=1/1, l=1/28
_расчёт матрицы H_: k=1/1, l=2/28
_расчёт матрицы H_: k=1/1, l=3/28
_расчёт матрицы H_: k=1/1, l=4/28
_расчёт матрицы H_: k=1/1, l=5/28
_расчёт матрицы H_: k=1/1, l=6/28
_расчёт матрицы H_: k=1/1, l=7/28
_расчёт матрицы H_: k=1/1, l=8/28
_расчёт матрицы H_: k=1/1, l=9/28
_расчёт матрицы H_: k=1/1, l=10/28
_расчёт матрицы H_: k=1/1, l=11/28
_расчёт матрицы H_: k=1/1, l=12/28
_расчёт матрицы H_: k=1/1, l=13/28
_расчёт матрицы H_: k=1/1, l=14/28
_расчёт матрицы H_: k=1/1, l=15/28
_расчёт матрицы H_: k=1/1, l=16/28
_расчёт матрицы H_: k=1/1, l=17/28
_расчёт матрицы H_: k=1/1, l=18/28
_расчёт матрицы H_: k=1/1, l=19/28
_расчёт матрицы H_: k=1/1, l=20/28
_расчёт матрицы H_: k=1/1, l=21/28
_расчёт матрицы H_: k=1/1, l=22/28
_расчёт матрицы H_: k=1/1,

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f371e991bd0>>
Traceback (most recent call last):
  File "/home/kodiak/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=3, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 1, 'cModel': '1 antenna', 'dN': 1, 'dModel': '1 antenna'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=2, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 1, 'cModel': '1 antenna', 'dN': 1, 'dModel': '1 antenna'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

###### <u>Наблюдаемость системы</u> без аэродинамики, без углового движения, 2 антенны - 1/4 монополь

In [9]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 2, 'cModel': '2 antennas', 'dN': 2, 'dModel': '2 antennas'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор состояния (изменённый):


Matrix([
[r_0^d_x(t)],
[r_0^d_z(t)],
[v_0^d_x(t)],
[v_0^d_z(t)]])

Количество кубсатов: 1
Количество чипсатов: 1

Неизвестные: n = 4 (на каждый чипсат по 4 параметров)
Известные: l = 4
∃ производные порядка k = 1.0 (Должна быть целой!)
Критерий (055): происзводные порядка 3.5



KeyboardInterrupt



In [10]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=2, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 2, 'cModel': '2 antennas', 'dN': 2, 'dModel': '2 antennas'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t), r_1^d_x(t), r_1^d_y(t), r_1^d_z(t), v_1^d_x(t), v_1^d_y(t), v_1^d_z(t)]])

Количество кубсатов: 1
Количество чипсатов: 2

Неизвестные: n = 12 (на каждый чипсат по 6 параметров)
Известные: l = 12
∃ производные порядка k = 1.0 (Должна быть целой!)
Критерий (055): происзводные порядка 7.5


KeyboardInterrupt: 

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=2, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 2, 'cModel': '2 antennas', 'dN': 2, 'dModel': '2 antennas'},
                                     hand_written_deriv=2, testprint=True, disp=False, Hcheck=False)

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 2, 'cModel': '2 antennas', 'dN': 2, 'dModel': '2 antennas'},
                                     hand_written_deriv=3, testprint=True, disp=False, Hcheck=False)

###### <u>Наблюдаемость системы</u> без аэродинамики, с угловым движением, антенны - (2) 1/4 монополь И (1) изотропный

In [21]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=1, 
                                     is_d_qw=True, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 1, 'cModel': '1 antenna', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), q_0^d_x(t), q_0^d_y(t), q_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t), ω_0^d_x(t), ω_0^d_y(t), ω_0^d_z(t)]])

Количество кубсатов: 1
Количество чипсатов: 1

Неизвестные: n = 12 (на каждый чипсат по 12 параметров)
Известные: l = 1
∃ производные порядка k = 12.0 (Должна быть целой!)
Критерий (055): происзводные порядка 90.0


Matrix([
[                                                                                                                     v_0^d_x(t)],
[                                                                                                                     v_0^d_y(t)],
[                                                                                                                     v_0^d_z(t)],
[                                                                                                              -2*w_0*v_0^d_z(t)],
[                                                                                                             -w_0**2*r_0^d_y(t)],
[                                                                                         3*w_0**2*r_0^d_z(t) + 2*w_0*v_0^d_x(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_x(t) - 0.5*q_0^d_y(t)*ω_0^d_z(t) + 0.5*q_0^d_z(t)*ω_0^d_y(t)],
[0.5*sqrt(-q_0^d_x(t)**2 - q_0^d_y(t)**2 - q_0^d_z(t)**2 + 1)*ω_0^d_y(t) +

__расчёт матрицы H_: k=1/12, l=1/1



KeyboardInterrupt



In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=3, 
                                     is_d_qw=True, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 1, 'cModel': '1 antenna', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=True)

###### <u>Проверка матрицы H на основе:</u> без аэродинамики, с угловым движением, антенны - 1/4 монополь

In [6]:
n_c=1
n_d=1
is_d_qw=True
gains={'cN': 0, 'cModel': 'isotropic', 'dN': 1, 'dModel': '1 antenna'}
is_drag=False
is_only_xz=False
testprint=False
hand_written_deriv=None
disp=False
r_d, v_d, q_d, ω_d = get_state_vector(func=kf.get_func, obj='d', n=n_d)
r_c, v_c, q_c, ω_c = get_state_vector(func=kf.get_func, obj='c', n=n_c)

o.c.n = n_c
o.f.n = n_d
o.f.r_orf, o.f.v_orf, o.f.q, o.f.w_brf = r_d, v_d, q_d, ω_d
o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_brf = r_c, v_c, q_c, ω_c

x = []
for i in range(n_d):
    x.extend(r_d[i])
    if is_d_qw:
        x.extend(q_d[i][1:4])
    x.extend(v_d[i])
    if is_d_qw:
        x.extend(v_d[i])
x = Matrix(x)

kf.my_print(f"Вектор состояния:", bold=True)
display(x.T)

o.v.GAIN_MODEL_C_N = gains['cN']
o.v.GAIN_MODEL_F_N = gains['dN']
o.v.init_choice_params()
o.c.gain_mode = o.v.GAIN_MODEL_C = gains['cModel']
o.f.gain_mode = o.v.GAIN_MODEL_F = gains['dModel']
o.v.NAVIGATION_ANGLES = is_d_qw
o.v.DYNAMIC_MODEL['aero drag'] = is_drag
o.v.MULTI_ANTENNA_TAKE = True
o.v.MULTI_ANTENNA_SEND = True

y, notes = kf.measure_antennas_power(c=o.c, f=o.f, v=o.v, p=o.p, j=int(len(x) // n_d), estimated_params=x, t=t)
if disp:
    kf.my_print(f"Вектор измерений:", bold=True)
    display(y)
    display(dω_d[0])

# >>>>>>>>>>>>>>>>> Проверка матрицы H <<<<<<<<<<<<<<<<<
H_1 = kf.h_matrix(t=t, v=o.v, f=o.f, c=o.c, r_f=r_d, r_c=r_c, 
                  q_f=[Matrix(q_d[i][1:4]) for i in range(o.f.n)], q_c=[Matrix(q_c[i][1:4]) for i in range(o.c.n)])
H_2 = zeros(*H_1.shape)
for ix, xx in enumerate(x):
    for iy, yy in enumerate(y):
        H_2[iy, ix] = yy.diff(xx)
tmp = H_1 - H_2

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), q_0^d_x(t), q_0^d_y(t), q_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

###### <u>Наблюдаемость системы</u> без аэродинамики, с угловым движением, 1 антенна - 1/4 монополь

In [5]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=1, 
                                     is_d_qw=True, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 1, 'cModel': '1 antenna', 'dN': 1, 'dModel': '1 antenna'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), q_0^d_x(t), q_0^d_y(t), q_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t), ω_0^d_x(t), ω_0^d_y(t), ω_0^d_z(t)]])

Количество кубсатов: 1
Количество чипсатов: 1

Неизвестные: n = 12 (на каждый чипсат по 12 параметров)
Известные: l = 1
∃ производные порядка k = 12.0 (Должна быть целой!)
Критерий (055): происзводные порядка 90.0
__расчёт матрицы H_: k=1/12, l=1/1



KeyboardInterrupt



In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=3, 
                                     n_d=1, 
                                     is_d_qw=True, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 1, 'cModel': '1 antenna', 'dN': 1, 'dModel': '1 antenna'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=2, 
                                     is_d_qw=True, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 1, 'cModel': '1 antenna', 'dN': 1, 'dModel': '1 antenna'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=1, 
                                     is_d_qw=True, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 1, 'cModel': '1 antenna', 'dN': 1, 'dModel': '1 antenna'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

###### <u>Наблюдаемость системы</u> с аэродинамикой, с угловым движением, 2 антенны - 1/4 монополь

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=1, 
                                     is_d_qw=True, 
                                     is_drag=True, 
                                     is_only_xz=True, 
                                     gains={'cN': 2, 'cModel': '2 antennas', 'dN': 2, 'dModel': '2 antennas'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), q_0^d_x(t), q_0^d_y(t), q_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t), ω_0^d_x(t), ω_0^d_y(t), ω_0^d_z(t)]])

Вектор состояния (изменённый):


Matrix([
[r_0^d_x(t)],
[r_0^d_z(t)],
[v_0^d_x(t)],
[v_0^d_z(t)]])

Количество кубсатов: 1
Количество чипсатов: 1

Неизвестные: n = 4 (на каждый чипсат по 4 параметров)
Известные: l = 4
∃ производные порядка k = 1.0 (Должна быть целой!)
Критерий (055): происзводные порядка 3.5


In [13]:
H, J, Jn, Δ, report = observe_system(n_c=3, 
                                     n_d=1, 
                                     is_d_qw=True, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 2, 'cModel': '2 antennas', 'dN': 2, 'dModel': '2 antennas'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), q_0^d_x(t), q_0^d_y(t), q_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Количество кубсатов: 3
Количество чипсатов: 1

Неизвестные: n = 12 (на каждый чипсат по 12 параметров)
Известные: l = 12
∃ производные порядка k = 1.0 (Должна быть целой!)
Критерий (055): происзводные порядка 7.5
_расчёт матрицы H_: k=1/1, l=1/12
_расчёт матрицы H_: k=1/1, l=2/12
_расчёт матрицы H_: k=1/1, l=3/12
_расчёт матрицы H_: k=1/1, l=4/12
_расчёт матрицы H_: k=1/1, l=5/12
_расчёт матрицы H_: k=1/1, l=6/12
_расчёт матрицы H_: k=1/1, l=7/12
_расчёт матрицы H_: k=1/1, l=8/12
_расчёт матрицы H_: k=1/1, l=9/12
_расчёт матрицы H_: k=1/1, l=10/12
_расчёт матрицы H_: k=1/1, l=11/12
_расчёт матрицы H_: k=1/1, l=12/12
Размерность матрицы H: (12, 1)
Размерность матрицы J: (12, 12)



KeyboardInterrupt



###### <u>Наблюдаемость системы</u> с аэродинамикой, с угловым движением, 3 антенны - 1/4 монополь

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=3, 
                                     is_d_qw=True, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 3, 'cModel': '3 antennas', 'dN': 3, 'dModel': '3 antenna'},
                                     hand_written_deriv=None, testprint=True, disp=False, Hcheck=False)

##### **<u>Согласно статье</u> 067 (Butcher E.A.) On Kalman Filtering and Observability in Nonlinear Sequential Relative Orbit Estimation**

In [4]:
from common_func import *

n_c, n_d = 1, 1
angles = True

o, num_params, t, w_0, μ, ρ, r_orb, v_orb = init_symbol_params()
o.v.SHAMANISM["ClohessyWiltshireC1=0"] = False
o.f.n = n_d
o.c.n = n_c
o.c.gain_mode = o.v.GAIN_MODEL_C = '2 antennas'
o.f.gain_mode = o.v.GAIN_MODEL_F = 'isotropic'
o.v.N_ANTENNA_C = 2
o.v.N_ANTENNA_F = 1
o.v.NAVIGATION_ANGLES = True
o.v.MULTI_ANTENNA_SEND = True
o.v.MULTI_ANTENNA_TAKE = True

o.init_classes()

o.f.r_orf, o.f.v_orf, o.f.q, o.f.w_irf = get_state_vector(func=kf.get_vars, obj='d', n=n_d)
o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_irf = get_state_vector(func=kf.get_vars, obj='c', n=n_c)
o.f.J = diag(*kf.get_vars("J", n=3))
o.v.dT_nav = var("dt")

F = Matrix(o.p.k.get_Phi(w0=w_0, w=o.f.w_irf))
kf.my_print(f"Матрица перехода:", bold=True)
display(F)

Параметры не могут быть загружены! Нет файла: kiamformation/data/config_choose.csv
Матрицы Ф:(6, 6), Q:(3, 3), P:(6, 6), D:(6, 3)
Высота орбиты: 400 км
Период орбиты: 1.54 часов
Плотность атмосферы: 6.404751331738951e-12 кг/м³
Матрицы Ф:(12, 12), Q:(6, 6), P:(12, 12), D:(12, 6)
Матрица перехода:


Matrix([
[1.0,          0,           0,   0,   0,   0,       dt,   0,         0,                                   0,                                  0,                                   0],
[  0,        1.0,           0,   0,   0,   0,        0,  dt,         0,                                   0,                                  0,                                   0],
[  0,          0,         1.0,   0,   0,   0,        0,   0,        dt,                                   0,                                  0,                                   0],
[  0,          0,           0, 1.0,   0,   0,        0,   0,         0,                              0.5*dt,                                  0,                                   0],
[  0,          0,           0,   0, 1.0,   0,        0,   0,         0,                                   0,                             0.5*dt,                                   0],
[  0,          0,           0,   0,   0, 1.0,        0,   0,         0,     

**Господи! Как тут получить матрицу через N итераций, ещё и умноженной на H!**

##### **<u>Влияние закрутки КА на наблюдаемость</u>**

###### <u>Инициализация</u>

In [1]:
from common_func import *

n_c, n_d = 1, 1
angles = True

o, num_params, t, w_0, μ, ρ, r_orb, v_orb = init_symbol_params()
o.f.n = n_d
o.c.n = n_c
o.f.r_orf, o.f.v_orf, o.f.q, o.f.w_irf = get_state_vector(func=kf.get_vars, obj='d', n=n_d)
o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_irf = get_state_vector(func=kf.get_vars, obj='c', n=n_c)
o.c.gain_mode = o.v.GAIN_MODEL_C = '2 antennas'
o.f.gain_mode = o.v.GAIN_MODEL_F = 'isotropic'
o.v.N_ANTENNA_C = 2
o.v.N_ANTENNA_F = 1
o.v.NAVIGATION_ANGLES = True
o.v.MULTI_ANTENNA_SEND = True
o.v.MULTI_ANTENNA_TAKE = True

Инициализация проекта kiam-formation
Параметры не могут быть загружены! Нет файла: kiamformation/data/config_choose.csv
Матрицы Ф:(6, 6), Q:(3, 3), P:(6, 6), D:(6, 3)
Высота орбиты: 400 км
Период орбиты: 1.54 часов
Плотность атмосферы: 6.404751331738951e-12 кг/м³


In [2]:
kf.measure_antennas_power(c=o.c, f=o.f, v=o.v, noise=0, produce=True, p=o.p, estimated_params=[])
o.v.MEASURES_VECTOR_NOTES
# o.v.MEASURES_VECTOR

['cf 0 0 0 0 2 1', 'cf 0 0 1 0 2 1']

###### <u>Исследование частных случаев</u> (без учёта динамики)

In [3]:
params_to_sub = []

# Движения в плоскости орбиты
params_to_sub += [(o.f.r_orf[0][1], 0), (o.c.r_orf[0][1], 0),
                  (o.f.v_orf[0][1], 0), (o.c.v_orf[0][1], 0),]

# Ориентация в плоскости орбиты
# params_to_sub += [(o.f.q[0][1], 0), (o.c.q[0][1], 0),
#                   (o.f.q[0][2], 0), (o.c.q[0][2], 0),]

# Кубсат неподвижен в ОСК
params_to_sub += [(o.c.r_orf[0][0], 0), (o.c.r_orf[0][1], 0), (o.c.r_orf[0][2], 0), 
                  (o.c.q[0][1], 0), (o.c.q[0][2], 0), (o.c.q[0][3], 0), ]

y = o.v.MEASURES_VECTOR.subs(params_to_sub)
y

Matrix([
[1.0*sqrt(r_0^d_x**2 + r_0^d_z**2)*(1.0*r_0^d_x**2 + 1.0*r_0^d_z**2)**(3/4)/Abs(r_0^d_x)**(3/2)],
[1.0*sqrt(r_0^d_x**2 + r_0^d_z**2)*(1.0*r_0^d_x**2 + 1.0*r_0^d_z**2)**(3/4)/Abs(r_0^d_z)**(3/2)]])

In [21]:
y1 = y[0].simplify()
y2 = y[1].simplify()

kf.my_print(f"Измерения:", bold=True)
display(y[0])
display(y[1])

kf.my_print(f"Чё то 1:", bold=True)
tmp = (1/y1**4 + 1/y2**4).simplify()
display(tmp)

kf.my_print(f"Чё то 2:", bold=True)
tmp = ((y1 / y2)**4).simplify()
display(tmp)

kf.my_print(f"Чё то 3:", bold=True)
t1 = 1/y1
t2 = 1/y2
tmp = (t1 / (t1**2 + t2**2)).simplify()
display(tmp)

Измерения:


1.0*sqrt(r_0^d_x**2 + r_0^d_z**2)*(1.0*r_0^d_x**2 + 1.0*r_0^d_z**2)**(3/4)/Abs(r_0^d_x)**(3/2)

1.0*sqrt(r_0^d_x**2 + r_0^d_z**2)*(1.0*r_0^d_x**2 + 1.0*r_0^d_z**2)**(3/4)/Abs(r_0^d_z)**(3/2)

Чё то 1:


0.999999999999999*(Abs(r_0^d_x**(3/2))**4 + Abs(r_0^d_z**(3/2))**4)/(r_0^d_x**2 + r_0^d_z**2)**5

Чё то 2:


1.0*r_0^d_z**6/r_0^d_x**6

Чё то 3:


1.0*(r_0^d_x**2 + r_0^d_z**2)**(5/4)*Abs(r_0^d_x**(3/2))/(Abs(r_0^d_x**(3/2))**2 + Abs(r_0^d_z**(3/2))**2)

*Та не получается ниче без динамики*

###### <u>Исследование частных случаев</u> (с учётом динамики)

In [3]:
params_to_sub = []

# Движения в плоскости орбиты
params_to_sub += [(o.f.r_orf[0][1], 0), (o.c.r_orf[0][1], 0),
                  (o.f.v_orf[0][1], 0), (o.c.v_orf[0][1], 0),]

# Ориентация в плоскости орбиты
# params_to_sub += [(o.f.q[0][1], 0), (o.c.q[0][1], 0),
#                   (o.f.q[0][2], 0), (o.c.q[0][2], 0),]

# Кубсат неподвижен в ОСК
params_to_sub += [(o.c.r_orf[0][0], 0), (o.c.r_orf[0][1], 0), (o.c.r_orf[0][2], 0), 
                  (o.c.q[0][1], 0), (o.c.q[0][2], 0), (o.c.q[0][3], 0), ]

# Динамика
c_f = kf.get_c_hkw(o.f.r_orf[0], o.f.v_orf[0], w_0)
r_f = kf.r_hkw(c_f, w_0, t)
params_to_sub += [(o.f.r_orf[0][0], r_f[0]), (o.f.r_orf[0][1], r_f[1]), (o.f.r_orf[0][2], r_f[2])]

y = o.v.MEASURES_VECTOR.subs(params_to_sub)
y

Matrix([
[1.0*sqrt((4*r_0^d_z + 2*v_0^d_x/ω_0 + v_0^d_z*sin(t*ω_0)/ω_0 + (-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0))**2 + (r_0^d_x + t*ω_0*(-24*r_0^d_z - 15*v_0^d_x/ω_0 - 6*v_0^d_z*sin(t*ω_0)/ω_0 - 6*(-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0)) + 2*v_0^d_z*cos(t*ω_0)/ω_0 - 2*v_0^d_z/ω_0 - (-24*r_0^d_z - 16*v_0^d_x/ω_0 - 6*v_0^d_z*sin(t*ω_0)/ω_0 - 6*(-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0))*sin(t*ω_0))**2)*(1.0*(4*r_0^d_z + 2*v_0^d_x/ω_0 + v_0^d_z*sin(t*ω_0)/ω_0 + (-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0))**2 + 1.0*(r_0^d_x + t*ω_0*(-24*r_0^d_z - 15*v_0^d_x/ω_0 - 6*v_0^d_z*sin(t*ω_0)/ω_0 - 6*(-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0)) + 2*v_0^d_z*cos(t*ω_0)/ω_0 - 2*v_0^d_z/ω_0 - (-24*r_0^d_z - 16*v_0^d_x/ω_0 - 6*v_0^d_z*sin(t*ω_0)/ω_0 - 6*(-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0))*sin(t*ω_0))**2)**(3/4)/Abs(r_0^d_x + t*ω_0*(-24*r_0^d_z - 15*v_0^d_x/ω_0 - 6*v_0^d_z*sin(t*ω_0)/ω_0 - 6*(-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0)) + 2*v_0^d_z*cos(t*ω_0)/ω_0 - 2*v_0^d_z/ω_0 - (-24*r_0^d_z - 16*v_0^d_x/ω_0 - 6*

**Симметрия относительно уравнений движения**

In [50]:
t1_n, t1_d = fraction(t1**2)  # Разделение на числитель и знаменатель
t2_n, t2_d = fraction(t2**2)

kf.my_print(f"Равные знаменатели:", bold=True)
display((t1_d - t2_d).simplify())

kf.my_print(f"Сумма числителей:", bold=True)
tmp_01 = ((t1_n + t2_n) ).simplify()
display(tmp_01)

Равные знаменатели:


0

Сумма числителей:


1.0*Abs((4*r_0^d_z*ω_0 + 2*v_0^d_x + v_0^d_z*sin(t*ω_0) - (3*r_0^d_z*ω_0 + 2*v_0^d_x)*cos(t*ω_0))/ω_0)**3 + 1.0*Abs((2*v_0^d_z*cos(t*ω_0) - 2*v_0^d_z + ω_0*(r_0^d_x - 3*t*(8*r_0^d_z*ω_0 + 5*v_0^d_x + 2*v_0^d_z*sin(t*ω_0) - (6*r_0^d_z*ω_0 + 4*v_0^d_x)*cos(t*ω_0))) + 2*(12*r_0^d_z*ω_0 + 8*v_0^d_x + 3*v_0^d_z*sin(t*ω_0) - (9*r_0^d_z*ω_0 + 6*v_0^d_x)*cos(t*ω_0))*sin(t*ω_0))/ω_0)**3